In [2]:
# DB에 저장할 데이터를 pulling하는 파일입니다.
import pandas as pd
import numpy as np
import FinanceDataReader as fdr
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings(action='ignore')

# 20211210 기준
std_date = '20211210'

date_list = [std_date]

date_list_for_PER = date_list

date_start = '20200102' # 1년의 데이터는 확보하기 위해서

N = 10 # 시가총액 상위 N 종목

df = pd.read_csv('./data/{}_total_stock.csv'.format(std_date), encoding='euc-kr')

# 시가총액 상위 N종목 코드 리스트
top_code = df.sort_values(by='시가총액', ascending=False).head(N).종목코드.to_list()

# 종목코드를 가지고 종목명을 찾을 수 있는 dictionary
code_to_name_dic = df[['종목코드', '종목명']].set_index('종목코드').to_dict()['종목명']

KOSPI = fdr.DataReader('KS11', date_start)

# 종목코드별 주가 데이터 dictionary에 저장
stock_price_dic = {}
for code in top_code:
    stock_price_dic[code] = fdr.DataReader(code, date_start)

In [6]:
print('date_add 함수 테스트 : {}'.format(date_add(date_start, -12)))
date_start

date_add 함수 테스트 : 2019-01-01 00:00:00


'20200102'

In [16]:
def date_add(date, month):
    '''date에 month만큼 더해준다. date는 20210102와 같은 꼴이나 datetime type으로 입력한다.'''
    if type(date) == str: # date가 datetime이 아니라 str 타입으로 들어온 경우
        date = datetime.strptime(date, '%Y%m%d')
    delta = timedelta(days=month*30.5)
    return date + delta


def earning_rate(stock_price : list, start_date, end_date):
    '''start_date부터 end_date까지 stock_price데이터를 가지고 수익률을 구해주는 함수(종가 기준)'''
    possible_start_date = stock_price[stock_price.index >= start_date].index[0]
    possible_end_date = stock_price[stock_price.index <= end_date].index[-1]
    
    start_price = stock_price.loc[possible_start_date].Close
    end_price = stock_price.loc[possible_end_date].Close
    
    return (end_price - start_price) / start_price

stock_price = stock_price_dic[top_code[0]]

n = 12 # n개월 간의 주가 추이 확인
# m = 6 # m개월 후의 수익률 확인

def ER_list_return(stock_price, n=n, date_list=date_list):
    '''stock_price정보를 가지고 base_date 기준 n개월 전, m개월 후 수익률(Earning Rate)을 구해준다.
    return : 데이터로 사용할 수익률, target을 구할 때 사용할 수익률, base_date'''
    earning_rate_list = []
    # target_list = []
    base_date_list = []  # 수익률 기준일
    
    # periods = 12//n # n = 2개월이면 12개월을 6개로 쪼개는 식
    for date in date_list:
        try:
        
            date_start = date_add(date, -12) # 수정
            date_base = date
            
            ER_before = earning_rate(stock_price, date_start, date_base)
            # ER_after = earning_rate(stock_price, date_end, date_add(date_end, m))
            
            earning_rate_list.append(ER_before)
            # target_list.append(ER_after)
            base_date_list.append(date_base)
            print(date_start, date_base, ER_before)
                
        except: # 종목코드가 바뀌거나 상장폐지돼서 주가 데이터가 없는 경우가 있는 것 같음. 이를 고려             
            break

    return earning_rate_list, base_date_list


# DataFrame 만들기 위해 모든 종목들에 대해 시행해주기
n_month_ER_data = []
# m_month_ER_data = []
date_data = []
code_data = []

for code in top_code:
    stock_price = stock_price_dic[code]
    earning_rate_list, base_date_list = ER_list_return(stock_price)
    n_month_ER_data += earning_rate_list
    # m_month_ER_data += target_list
    date_data += base_date_list
    code_data += [code]*len(base_date_list)

data = pd.DataFrame({
                     'date' : date_data,
                     'code' : code_data,
                     f'{n}개월 간 수익률': n_month_ER_data,
                    #  f'{m}개월 후 수익률' : m_month_ER_data
                    })

data['name'] = data.code.apply(lambda x: code_to_name_dic[x])

KOSPI_n_earning, KOSPI_base_date = ER_list_return(KOSPI)

KOSPI_data = pd.DataFrame({
                     'date' : KOSPI_base_date,
                     f'KOSPI {n}개월 간 수익률': KOSPI_n_earning,
                    #  f'KOSPI {m}개월 후 수익률' : KOSPI_m_earning
                    })

data = data.merge(KOSPI_data, on='date')

# KOSPI 대비 수입률 Feature 만들기
data[f'KOSPI대비 {n}개월 간 수익률'] = data[f'{n}개월 간 수익률'] - data[f'KOSPI {n}개월 간 수익률']
# data[f'KOSPI대비 {m}개월 후 수익률'] = data[f'{m}개월 후 수익률'] - data[f'KOSPI {m}개월 후 수익률']
data[f'{n}개월 간 KOSPI 이김'] = data[f'KOSPI대비 {n}개월 간 수익률'].apply(lambda x: 1 if x>=0 else 0)
# data[f'KOSPI보다 많이 오름'] = data[f'KOSPI대비 {m}개월 후 수익률'].apply(lambda x: 1 if x>=0 else 0)



# PER 파일 불러와서 dic에 저장
df_PER_dic = {}
for date in date_list_for_PER:
    # 결측치는 0으로
    df_PER_dic[date] = pd.read_csv('./data/{}_PER_PBR.csv'.format(date), encoding='euc-kr')
    df_PER_dic[date][['PER', 'PBR']] = df_PER_dic[date][['PER', 'PBR']].fillna(10000)
    df_PER_dic[date][['배당수익률']] = df_PER_dic[date][['배당수익률']].fillna(0)

def near_date_for_PER(date, date_list=date_list_for_PER):
    '''한국거래소에서 받은 PER를 데이터 누수(미래의 PER 사용;) 없이 사용하기 위해 date에 따라 적절한 사용가능 날짜를 date_list에서 뽑아 리턴.
    date는 datetime 타입'''
    for d in date_list:
        d = datetime.strptime(d, '%Y%m%d')
        if d - timedelta(days=30) <= date < d + timedelta(days=336):
            return d.strftime('%Y%m%d')
    return date_list[0]

# near_date_for_PER(data.date[0])

def search_for_PER(code, date):
    '''code와 date 조건에 맞는 데이터를 PER가 담긴 df에서 찾아줌'''
    if type(date) != str:
        date = near_date_for_PER(date)
    df = df_PER_dic[date]
    cond = df['종목코드'] == code
    
    if cond.sum() == 0:
        fake_data = [0] * len(df.columns)
        return pd.DataFrame([fake_data], columns=df.columns)
    return df[cond]

# search_for_PER('069117', '20110102')

temp_df_list = []
for date, code in zip(data.date, data.code):
    temp_df_list.append(search_for_PER(code, date))
data_PER = pd.concat(temp_df_list, ignore_index=True)

data = pd.concat([ data, data_PER[['PER', 'PBR', '배당수익률']] ], axis=1)
data['PER_inv'] = 1/data['PER']
data['PBR_inv'] = 1/data['PBR']

data.head(10)

2020-12-09 00:00:00 20211210 0.04059539918809202
2020-12-09 00:00:00 20211210 0.0
2020-12-09 00:00:00 20211210 0.3715277777777778
2020-12-09 00:00:00 20211210 0.09610705596107055
2020-12-09 00:00:00 20211210 0.05530642750373692
2020-12-09 00:00:00 20211210 0.6275176701918478
2020-12-09 00:00:00 20211210 -0.11483253588516747
2020-12-09 00:00:00 20211210 0.25265017667844525
2020-12-09 00:00:00 20211210 0.0835509138381201
2020-12-09 00:00:00 20211210 0.34065934065934067
2020-12-09 00:00:00 20211210 0.09245609641912278


,date,code,12개월 간 수익률,name,KOSPI 12개월 간 수익률,KOSPI대비 12개월 간 수익률,12개월 간 KOSPI 이김,PER,PBR,배당수익률,PER_inv,PBR_inv
0,20211210,005930,0.040595,삼성전자,0.092456,-0.051861,0,20.02,1.95,3.89,0.049950,0.512821
1,20211210,000660,0.000000,SK하이닉스,0.092456,-0.092456,0,17.33,1.69,0.97,0.057703,0.591716
2,20211210,035420,0.371528,NAVER,0.092456,0.279072,1,57.44,8.81,0.10,0.017409,0.113507
3,20211210,207940,0.096107,삼성바이오로직스,0.092456,0.003651,1,247.39,12.96,0.00,0.004042,0.077160
4,20211210,005935,0.055306,삼성전자우,0.092456,-0.037150,0,10000.00,10000.00,4.24,0.000100,0.000100
5,20211210,035720,0.627518,카카오,0.092456,0.535062,1,331.98,8.57,0.02,0.003012,0.116686
6,20211210,051910,-0.114833,LG화학,0.092456,-0.207289,0,111.01,3.21,1.35,0.009008,0.311526
7,20211210,006400,0.252650,삼성SDI,0.092456,0.160194,1,82.51,3.85,0.14,0.012120,0.259740
8,20211210,005380,0.083551,현대차,0.092456,-0.008905,0,38.05,0.83,1.45,0.026281,1.204819
9,20211210,000270,0.340659,기아,0.092456,0.248203,1,23.02,1.16,1.17,0.043440,0.862069


In [9]:
stock_price

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,44100,44300,42300,42500,1355029,-0.040632
2020-01-03,42600,42900,41500,42000,1569720,-0.011765
2020-01-06,41350,41950,41350,41900,830880,-0.002381
2020-01-07,41900,42300,41550,42050,1124218,0.003580
2020-01-08,41300,41900,41100,41300,997335,-0.017836
...,...,...,...,...,...,...
2021-12-07,83400,84200,82700,83300,1436180,-0.007151
2021-12-08,84100,84300,83200,83200,1343368,-0.001200
2021-12-09,82400,84800,82400,84100,1774961,0.010817


In [11]:
stock_price_dic['005930'].head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-12-01,67100,68300,67100,67800,22327173,0.016492
2020-12-02,68400,69900,68300,69500,29703942,0.025074
2020-12-03,70100,70500,69300,69700,23677989,0.002878
2020-12-04,70400,72100,70100,71500,28130533,0.025825
2020-12-07,72400,73500,71900,72900,28490496,0.019580


In [9]:
top_code

['005930',
 '000660',
 '035420',
 '207940',
 '005935',
 '035720',
 '051910',
 '006400',
 '005380',
 '000270']

In [10]:
for code in top_code:
    print(code_to_name_dic[code])

삼성전자
SK하이닉스
NAVER
삼성바이오로직스
삼성전자우
카카오
LG화학
삼성SDI
현대차
기아
